pip install transformers torchvision pillow

need to clone the OFA and OFA-base from github:
git clone --single-branch --branch feature/add_transformers https://github.com/OFA-Sys/OFA.git
pip install OFA/transformers/
git clone https://huggingface.co/OFA-Sys/OFA-base 

Also, might need to install git-lfs
Then in the cloned OFA-base folder:
git lfs install
git lfs pull


In [1]:
import torch
from PIL import Image
from torchvision import transforms
from transformers import OFATokenizer, OFAModel # OFAModel only works in older version of transformers (from pip install OFA/transformers/)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

In [ ]:
ckpt_dir = "/Users/junwu/Documents/ECS270/Project/Image-Model-with-Grounding-DINO/OFA-base"  # path to local OFA-base directory
tokenizer = OFATokenizer.from_pretrained(ckpt_dir)
model = OFAModel.from_pretrained(ckpt_dir) # .to(device)
model.eval()

# Image preprocessing 
resolution = 384
mean, std = [0.5] * 3, [0.5] * 3
transform = transforms.Compose([
    lambda image: image.convert("RGB"),
    transforms.Resize((resolution, resolution), interpolation=Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

#  Load and process image 
image = Image.open("/Users/junwu/Documents/ECS270/Project/Image-Model-with-Grounding-DINO/input-data/clock.jpg")  
patch_img = transform(image).unsqueeze(0)  # shape: (1, 3, H, W)

# Prepare text prompt 
question = " what does the image describe?"
inputs = tokenizer(question, return_tensors="pt").input_ids

# Generate caption 
outputs = model.generate(
    input_ids=inputs,
    patch_images=patch_img,
    patch_masks=torch.tensor([True]),
    num_beams=5,
    max_length=50,
    no_repeat_ngram_size=3,
)

# Decode and print result 
caption = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print("Generated caption:", caption[0])

caption = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
with open("OFA_output.txt", "w") as f:
    f.write(caption)

/Users/junwu/Documents/ECS270/Project/Image-Model-with-Grounding-DINO/OFA-base
<super: <class 'OFATokenizer'>, <OFATokenizer object>>
Generated caption:  wall clock on a white background


## COCO Evaluation

In [3]:
import json

coco_dataset = open("../instances_val2017.json")
coco_json = json.load(coco_dataset)
coco_images = coco_json["images"]
coco_annotations = coco_json["annotations"]
coco_categories = coco_json["categories"]
input_url = []
image_url = []
for anno in coco_annotations:
    for images in coco_images:
        if anno["image_id"] == images["id"]:
            image_url.append(images["coco_url"])
            break
    for categories in coco_categories:
        if anno["category_id"] == categories["id"]:
            input_url.append("../input-data/" + categories["name"] + ".jpg")
            break

In [4]:
print(len(input_url))
print(len(image_url))

36781
36781


In [5]:
ckpt_dir = "/Users/junwu/Documents/ECS270/Project/Image-Model-with-Grounding-DINO/OFA-base"  # path to local OFA-base directory
tokenizer = OFATokenizer.from_pretrained(ckpt_dir)
model = OFAModel.from_pretrained(ckpt_dir) # add .to(devide)
model.eval()

open("OFA_output.txt", "w").close() # clear the text file

# Image preprocessing 
resolution = 384
mean, std = [0.5] * 3, [0.5] * 3
transform = transforms.Compose([
    lambda image: image.convert("RGB"),
    transforms.Resize((resolution, resolution), interpolation=Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

full_results = []
for x in range(len(input_url)):
    #  Load and process image 
    image = Image.open(input_url[x])  
    patch_img = transform(image).unsqueeze(0)  # shape: (1, 3, H, W)

    # Prepare text prompt 
    question = " what does the image describe?"
    inputs = tokenizer(question, return_tensors="pt").input_ids

    # Generate caption 
    outputs = model.generate(
        input_ids=inputs,
        patch_images=patch_img,
        patch_masks=torch.tensor([True]),
        num_beams=5,
        max_length=50,
        no_repeat_ngram_size=3,
    )

    # Decode and print result 
    caption = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    print("Generated caption:", caption)

    image_id = coco_annotations[x]["image_id"]
    category_id = coco_annotations[x]["category_id"]

    with open("OFA_output.txt", "a") as f:
         f.write(f"{image_url[x]} || {caption} || {image_id} || {category_id}\n")

/Users/junwu/Documents/ECS270/Project/Image-Model-with-Grounding-DINO/OFA-base
<super: <class 'OFATokenizer'>, <OFATokenizer object>>


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:4316.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Generated caption:  portrait of a golden retriever dog on a white background
Generated caption:  portrait of a golden retriever dog on a white background
Generated caption:  portrait of a golden retriever dog on a white background
Generated caption:  portrait of a golden retriever dog on a white background
Generated caption:  portrait of a golden retriever dog on a white background


KeyboardInterrupt: 